# Sequence Analysis with Python

Author: Riccardo Maria Pesce (riccardompesce@gmail.com)

Tutor: Veli Makinen (veli.makinen@helsinki.fi)

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [58]:
from collections import defaultdict
from itertools import product

import numpy as np
from numpy.random import choice

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [59]:
def dna_to_rna(s):
    to_dict = {"T": "U", "A": "A", "G": "G", "C": "C"}

    return "".join([to_dict[ch] for ch in s])
    
if __name__ == '__main__':
    print(dna_to_rna("AACGTGATTTC"))

AACGUGAUUUC


### Idea of solution

Trascribing a DNA sequence into RNA can be achieved by replacing each "T" (thymine) with "U" (uracil). As the problem requested, let's use a `dict` to store DNA-RNA correspondences, and with the aid of the `join` method, we can build up the RNA string, by first translating each single character in the `s` string (which represents 
our DNA sequence) into the corresponding character of the 
resulting RNA sequence.

### Discussion

Our function works as expected and this can also be seen either with the unit tests or by using more well known and 
established DNA-to-RNA converters out there.

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [60]:
# Importing necessary libraries to implement get_dict()
import urllib, re

from bs4 import BeautifulSoup

def save_html_page(url):
    """
    Given an url, it download a web page locally (in html format)
    """
    urllib.request.urlretrieve(url, "page.html")

def make_dict():
    # Parsing the data and turning it into a BeautifulSoup object
    soup = BeautifulSoup(open("page.html"), "html.parser")

    # Parsing the actual content we are interested, i.e. the table between <pre></pre> tags
    pre_content = soup.find("pre")
    text_content = pre_content.text.strip()

    # Extracting the informations we are interested in
    dna_seqs = re.findall(r"[A-Z][A-Z][A-Z]", text_content)
    amino_acids = re.findall(r" ([A-Z]|[*]) ", text_content)

    # Returning the dict result
    return dict(zip(dna_seqs, amino_acids))

def get_dict():
    return {'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}
    
if __name__ == '__main__':
    file_name = "page.html"
    url = "http://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N"

    # Run once to download the web page locally
    # save_html_page(url)

    print(make_dict())

{'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}


### Idea of solution
The task could be accomplished in different ways:
- By using directly a request object within the `BeautifulSoup` class. 
- By using a web page previously downloaded locally.

The approach I chose was the second, even though the first was in our first iteration. After making the parsing and creating the dictionary with the conversion rules, I printed the result, copied the final dictionary and returned it as a literal in the `get_dict()` module, so as to keep it compliant with the tests and not to make any request from within the internal server.

### Discussion
The solution works as expected. It would have been better if we could make an "on-demand" solution which scrapes the page on request and returns the dictionary from within the module `get_dict()` itself. However, that would violate a web-scraping etiquette and good practice which discourages making too many requests to the server: it would cause too much useless traffic. While we could make a lazy execution, that is, requesting the page only once, cache it, and use the local copy as our object, these techniques are out of the scope of this project, but can be found in later projects where I would integrate requests to web API endpoints to gather data. 

3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [61]:
def get_dict_list():
    d = get_dict()
    reverse = {}

    for v, k in d.items():
        if k not in reverse.keys():
            reverse[k] = []

        reverse[k].append(v)

    return reverse
    
if __name__ == '__main__':
    aa_to_codons = get_dict_list()
    print(aa_to_codons)

{'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UGA', 'UAG'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}


### Idea of solution

The idea is simple: reversing the dictionary, but each value will be a list, since an amino acid can be a translation of different codons. The code is quite clear: for each key-value pair, if value is not present in the dictionary we want to return, we create that entry with an empty list which will later be filled.

### Discussion

The succesful tests show us the code works as intended.

With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [62]:

def rna_to_prot(s):
    d = get_dict()

    # Let's split s every 3 character so we can use our dict to convert each single sequence, and then we will join it
    splitted = [s[i:i + 3] for i in range(0, len(s), 3)]

    return "".join([d[seq] for seq in splitted])

def dna_to_prot(s):
    return rna_to_prot(dna_to_rna(s))

if __name__ == '__main__':
    print(dna_to_prot("ATGATATCATCGACGATGTAG"))

MISSTM*


### Idea of solution

The idea is to split the sequence `s` into sub-sequences of length 3 each, and map these sub-sequences using the dictionary returned by `get_dict()` into proteins.

### Discussion

The code does work and no other thing is worth adding at this point.

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function.

In [63]:
import pandas as pd

def make_probability_dict():
    # Parsing the data and turning it into a BeautifulSoup object
    page_url = "http://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N"

    page = urllib.request.urlopen(page_url)
    soup = BeautifulSoup(page, "html.parser")

    # Parsing the actual content we are interested, i.e. the table between <pre></pre> tags
    pre_content = soup.find("pre")
    text_content = pre_content.text.strip()

    # Extracting the informations we are interested in
    dna_seqs = re.findall(r"[A-Z][A-Z][A-Z]", text_content)
    aa = re.findall(r" ([A-Z|\*]) ", text_content)
    fract = re.findall(r"[A-Z|\*] (\d\.\d+)", text_content)
    freqs = re.findall(r" (\d+\.\d) \(", text_content)
    nums = re.findall(r"\(\s*(\d+)\)", text_content)

    # Turning fract and freq to numerical values
    fract = [float(f) for f in fract]
    freqs = [float(f) for f in freqs]
    nums = [float(f) for f in nums]

    dna_to_nums = dict(zip(dna_seqs, nums))
    aa_to_nums = pd.DataFrame(zip(aa, nums), columns=["aa", "nums"]).groupby("aa").nums.sum().to_dict()
    dna_to_aa = dict(zip(dna_seqs, aa))
    
    probs = {}
    for dna, num in dna_to_nums.items():
        associated_aa = dna_to_aa[dna]
        total_by_aa = aa_to_nums[associated_aa]
        probs[dna] = num / total_by_aa

    return probs

def get_probabability_dict():
    return {'UUU': 0.4641342698782968, 'UCU': 0.18758584014344437, 'UAU': 0.443338109266921, 'UGU': 0.45615733050366836, 'UUC': 0.5358657301217032, 'UCC': 0.21795953165920925, 'UAC': 0.5566618907330789, 'UGC': 0.5438426694963316, 'UUA': 0.07656764558436285, 'UCA': 0.15051714801827132, 'UAA': 0.29701911804823383, 'UGA': 0.46624296805302623, 'UUG': 0.1290578537068707, 'UCG': 0.05439771371883908, 'UAG': 0.23673791389873997, 'UGG': 1.0, 'CUU': 0.13171591206484023, 'CCU': 0.2867313296570278, 'CAU': 0.4185152128433691, 'CGU': 0.08010752804820104, 'CUC': 0.1955768259144855, 'CCC': 0.3234703981288551, 'CAC': 0.5814847871566309, 'CGC': 0.18377662978978224, 'CUA': 0.0713801723134756, 'CCA': 0.2766025276376192, 'CAA': 0.26501675921017337, 'CGA': 0.1088124833207855, 'CUG': 0.39570159041596514, 'CCG': 0.11319574457649788, 'CAG': 0.7349832407898266, 'CGG': 0.20155434006721587, 'AUU': 0.36107219301206106, 'ACU': 0.2467688440166039, 'AAU': 0.4703669907468028, 'AGU': 0.14960182300967595, 'AUC': 0.4698662895003286, 'ACC': 0.35523154074391966, 'AAC': 0.5296330092531971, 'AGC': 0.23993794345056002, 'AUA': 0.16906151748761036, 'ACA': 0.28418772983891855, 'AAA': 0.43404935110207155, 'AGA': 0.21465774794262568, 'AUG': 1.0, 'ACG': 0.11381188540055791, 'AAG': 0.5659506488979285, 'AGG': 0.21109127083138968, 'GUU': 0.18177011180348712, 'GCU': 0.26592161421413735, 'GAU': 0.4645424191930427, 'GGU': 0.1630865131632947, 'GUC': 0.23830637956135173, 'GCC': 0.39978112134364696, 'GAC': 0.5354575808069573, 'GGC': 0.33710935809444503, 'GUA': 0.11657741053350681, 'GCA': 0.2281212631716276, 'GAA': 0.42245266280361615, 'GGA': 0.24992165149690412, 'GUG': 0.46334609810165434, 'GCG': 0.10617600127058811, 'GAG': 0.5775473371963838, 'GGG': 0.24988247724535617}
    
if __name__ == '__main__':
    codon_to_prob = make_probability_dict()
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i*6:6+i*6]
        ))

AAA: 0.434049	AAC: 0.529633	AAG: 0.565951	AAU: 0.470367	ACA: 0.284188	ACC: 0.355232
ACG: 0.113812	ACU: 0.246769	AGA: 0.214658	AGC: 0.239938	AGG: 0.211091	AGU: 0.149602
AUA: 0.169062	AUC: 0.469866	AUG: 1.000000	AUU: 0.361072	CAA: 0.265017	CAC: 0.581485
CAG: 0.734983	CAU: 0.418515	CCA: 0.276603	CCC: 0.323470	CCG: 0.113196	CCU: 0.286731
CGA: 0.108812	CGC: 0.183777	CGG: 0.201554	CGU: 0.080108	CUA: 0.071380	CUC: 0.195577
CUG: 0.395702	CUU: 0.131716	GAA: 0.422453	GAC: 0.535458	GAG: 0.577547	GAU: 0.464542
GCA: 0.228121	GCC: 0.399781	GCG: 0.106176	GCU: 0.265922	GGA: 0.249922	GGC: 0.337109
GGG: 0.249882	GGU: 0.163087	GUA: 0.116577	GUC: 0.238306	GUG: 0.463346	GUU: 0.181770
UAA: 0.297019	UAC: 0.556662	UAG: 0.236738	UAU: 0.443338	UCA: 0.150517	UCC: 0.217960
UCG: 0.054398	UCU: 0.187586	UGA: 0.466243	UGC: 0.543843	UGG: 1.000000	UGU: 0.456157
UUA: 0.076568	UUC: 0.535866	UUG: 0.129058	UUU: 0.464134


### Idea of solution

The function will be similar to `get_dict()` with the only difference being the field we are parsing this time, which is the frequency in the fourth column. We will map the parsed probabilities into `float`, since they're initially parsed into strings.
The function will then calculate the relative (for each amino-acid) percentages one the normalized frequencies. Once that is finished, the probabilities are calculated for each amino.

### Discussion

As we can see from the output, the function worked perfectly.

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [64]:
class ProteinToMaxRNA:
    
    def __init__(self):
        pass

    def convert(self, s):
        prob_dict = get_probabability_dict()
        prot_to_rna = get_dict_list()

        most_prob_seq = []

        for prot in s:
            prob_seqs = prot_to_rna[prot]
            most_prob = prob_seqs[0]

            for seq in prob_seqs:
                if prob_dict[seq] > prob_dict[most_prob]:
                    most_prob = seq

            most_prob_seq.append(most_prob)

        return "".join(most_prob_seq)


if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print(protein_to_rna.convert("LTPIQNRA"))

CUGACCCCCAUCCAGAACAGAGCC


### Idea of solution

The class method `convert()` maps each protein sequence into the most likely RNA sequence by first getting the probabilities and protein-to-rna dictionaries, and then for each protein in the sequence s, we get the one with the highest probability.

### Discussion

Using the [translation tool](http://biomodel.uah.es/en/lab/cybertory/analysis/trans.htm), we translate the result given by the function, and we can see it is exactly the protein sequence given.



Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [65]:
def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    cum_dist = {}
    start = 0.00

    for event, prob in dist.items():
        cum_dist[event] = [start, prob + start]
        start = start + prob

    r = np.random.uniform()

    for event in cum_dist.keys():
        if r >= cum_dist[event][0] and r < cum_dist[event][1]:
            return event

    

if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(", ".join(random_event(distribution) for _ in range(12)))

A, C, T, A, C, C, T, C, T, T, A, T


### Idea of solution

The function will first calculate the cumulative distribution, including for each protein the beginning and the end of the "bin" to which such protein belongs to. Then samples uniformly a random number from 0 to 1, and check in which bin it falls, returning the relative protein.

### Discussion

The function samples according to the proteins own probabilities, therefore it is correct

With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [66]:
class ProteinToRandomRNA(object):
    
    def __init__(self):
        pass

    def convert(self, s):
        probs_dict = get_probabability_dict()
        prot_to_rna = get_dict_list()

        rand_seq = []

        for prot in s:
            translations = prot_to_rna[prot]
            probs = [probs_dict[seq] for seq in translations]
            
            dist = dict(zip(translations, probs))

            rand_e = random_event(dist)

            if rand_e == None:
                print(s)
                print(dist)

            rand_seq.append(rand_e)

        return "".join(rand_seq)
        
if __name__ == '__main__':
    protein_to_random_codons = ProteinToRandomRNA()
    print(protein_to_random_codons.convert("LTPIQNRA"))

CUCACGCCUAUUCAGAACCGAGCU


### Idea of solution

The method is similiar to the previous one, with the sole exception that instead of using the most probable RNA, the function uses the previously defined random event to sample from the cumulative distribution of each RNA.

### Discussion

The function does the correct sampling, as also assessed by the unit tests.

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [67]:

def sliding_window(s, k):
    """
    This function returns a generator that can be iterated over all
    starting position of a k-window in the sequence.
    For each starting position the generator returns the nucleotide frequencies
    in the window as a dictionary.
    """
    i = 0
    unique = set(s)

    while i + k < len(s):
        sub = s[i:i + k]
        freq = {n: sub.count(n) for n in unique}
        yield freq
        i += 1
    
if __name__ == '__main__':
    s = "TCCCGACGGCCTTGCC"
    #for d in sliding_window(s, 4):
    #    print(d)
    for d in sliding_window("ACGTTGCA", 4):
        print(d)

{'A': 1, 'C': 1, 'G': 1, 'T': 1}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 0, 'G': 2, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}


### Idea of solution

For each substring of size `k`, the function is going to compute how many times each unique nucletoide is present, and will store this information in a dictionary, whose key is the nucletoide and the value is the frequency.

### Discussion

The functions shows the correct frequencies for each window. It would seem that, with the number of unique characters increasing we would get sparse representations. But this is how the problem is structured in its domain.

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [68]:
def context_list(s, k):
    i = 0
    contextes = {}

    while i + k < len(s):
        context = s[i:i + k]
        if context in contextes.keys():
            contextes[context] = "".join(list(contextes[context]) + [s[i + k]])
        else:
            contextes[context] = s[i + k]
        i += 1

    return contextes
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATCTAG"
    d = context_list(s, k)
    print(d)

{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}


### Idea of solution

The idea is to iterate through each k-mer, which will be our context, and for each of these, we add in a dictionary where keys are the iterated contextes, the following symbol appearing after it.

### Discussion

The function passes the unit tests, therefore it works as expected.

11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [69]:
def context_probabilities(s, k):
    context_probs = {}

    for ctx, val in context_list(s, k).items():
        context_probs[ctx] = {ch: val.count(ch) / len(val) for ch in val}

    return context_probs

    
if __name__ == '__main__':
    k = 0
    s = "ATGATATCATCGACGATGTAG"
    print(context_probabilities(s, k))
    k = 2
    print(context_probabilities(s, k))

{'': {'A': 0.3333333333333333, 'T': 0.2857142857142857, 'G': 0.23809523809523808, 'C': 0.14285714285714285}}
{'AT': {'G': 0.4, 'A': 0.2, 'C': 0.4}, 'TG': {'A': 0.5, 'T': 0.5}, 'GA': {'T': 0.6666666666666666, 'C': 0.3333333333333333}, 'TA': {'T': 0.5, 'G': 0.5}, 'TC': {'A': 0.5, 'G': 0.5}, 'CA': {'T': 1.0}, 'CG': {'A': 1.0}, 'AC': {'G': 1.0}, 'GT': {'A': 1.0}}


### Idea of solution

This module counts the total number of contextes together with contextes themselves and then it converts these frequencies into probabilities, by dividing the number of occurrences of each element by the total number of contextes.

### Discussion

The function does indeed turns frequencies into the right probabilities as the tests results show. We can also see how powerful Python's `dict` type is.

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [70]:
class MarkovChain:
    
    def __init__(self, zeroth, kth, k=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def generate(self, n, seed=None):
        np.random.seed(seed)
        
        seq = []
        i = 0

        while i < self.k and i < n:
                seq.append(random_event(self.zeroth))
                i += 1

        while i < n:
            ctx = "".join(seq[i - self.k:i])
            seq.append(random_event(self.kth[ctx]))
            
            i += 1

        return "".join(seq)
            


if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
    n = 10
    seed = 0
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n, seed))

TGTATGATGA


### Idea of solution

The function is required to initialize the given seed and then generate the first k element using the zeroth dictionary, before generating the rest using the given kth dictionary of dictionaries.

### Discussion

The function passes the unit tests. In order not to go out of the given length `n`, we have to add this check during the initialization.

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [71]:
from itertools import product

def context_pseudo_probabilities(s, k):
    context_pseudo_probs = {}
    
    symbols = "ACGT"
    contexts = context_list(s, k)

    k_mers = ["".join(t) for t in product(symbols, repeat=k)]

    for k_mer in k_mers:
        if k_mer not in contexts:
            contexts[k_mer] = symbols
        else:
            contexts[k_mer] += symbols

    for k_mer in k_mers:
        ctx = contexts[k_mer]
        context_pseudo_probs[k_mer] = {ch: ctx.count(ch) / len(ctx) for ch in symbols}

    return context_pseudo_probs

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth = context_pseudo_probabilities(s, k)
    zeroth = context_pseudo_probabilities(s, 0)[""]
    print(f"zeroth: {zeroth}")
    print("\n".join(f"{k}: {dict(v)}" for k, v in kth.items()))
    
    print("\n", MarkovChain(zeroth, kth, k).generate(20))

zeroth: {'A': 0.32, 'C': 0.16, 'G': 0.24, 'T': 0.28}
AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666

### Idea of solution

In order to initialize, we define a variable `contextes`, which will be assigned to the function call `context_list()`, with the sole exception that we add to the already present values having as the preceeding letters the ones in the context, the symbols used, `"ACGT"`, so as to initialize the frequencies as uniform at beginning, and then of course change them as needed as we go along the input we get.

### Discussion

The function output is coherent to the description, and the test have been successful. As said above, we add to every context in `contextes` the string `"ACGT"` so as to initialize them.

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [72]:
class MarkovProb:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def probability(self, s):
        prob = 1.0
        i = 0

        while i < self.k and i < len(s):
            prob *= self.zeroth[s[i]]
            i += 1

        while i < len(s):
            context = s[i - self.k:i]
            prob *= self.kth[context][s[i]]
            i += 1

        return prob

    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovProb(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Probability of sequence {s} is {mc.probability(s)}")

Probability of sequence ATGATATCATCGACGATGTAG is 2.831270190340017e-10


### Idea of solution

Given a sequence, it's zero order and k-order context probability dictionaries, we have to compute the compound probability (obtained by multiplying the probabilities of every symbol given its context).

### Discussion

The function seems to return the correct probability as it is also tested by the unit tests. 

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [73]:
class MarkovLog(object):
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def log_probability(self, s):
        prob = 1.0
        i = 0

        while i < self.k and i < len(s):
            prob *= self.zeroth[s[i]]
            i += 1

        while i < len(s):
            context = s[i - self.k:i]
            prob *= self.kth[context][s[i]]
            i += 1

        return np.log2(prob)
    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovLog(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {s} is {mc.log_probability(s)}")

Log probability of sequence ATGATATCATCGACGATGTAG is -31.717831515538307


### Idea of solution

The function is the same as the previously defined `Markov`'s `probability`, except that we return the logarithm of it.

### Discussion

The function returns the correct log probability, which should go from minus infinity to zero. Being the probabilities as calculated in the previous module very low, in order to avoid possible numerical underflows, using log probabilities help us getting numbers which do not present such risk.

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [74]:
from collections import Counter

def better_context_probabilities(s, k):
    better_context_probs = {}
    
    symbols = "ACGT"
    contexts = context_list(s, k)

    k_mers = ["".join(t) for t in product(symbols, repeat=k)]

    for k_mer in k_mers:
        ctx = contexts.get(k_mer, "")
        better_context_probs[k_mer] = {ch: (ctx + symbols).count(ch) / len(ctx + symbols) for ch in symbols}

    return better_context_probs

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = better_context_probabilities(s, k)
    print("\n".join(f"{k}: {v}" for k, v in d.items()))

AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666}
TG: {'A': 0.3333333333333333, 'C': 0.16666666666666

### Idea of solution

The module is exactly the same as defined at `e07.13` with the only exception that we are not concatenating our symbols directly to our contextes.

### Discussion

Results are corrected as shown by the tests. We achieved a memory efficient method of computing the probabilities.

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [75]:
class SimpleMarkovChain(object):
    def __init__(self, s, k):
        self.s = s
        self.k = k

    def generate(self, n, seed=None):
        np.random.seed(seed)

        a = list(set(self.s))
        p = [f / len(self.s) for f in Counter(self.s).values()]

        return "".join(np.random.choice(a=a, size=n, p=p))
        
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    n = 10
    seed = 7
    mc = SimpleMarkovChain(s, k)
    print(mc.generate(n, seed))

ACGCTGGAAG


### Idea of solution

Instead of concatenating, we compute the frequency of each symbol and use that to predict the sequence through the suggested module `np.random.choice()`. Therefore, there is no explicit context we refer to, just the concatenation of symbols `s`.

### Discussion

The module passes all the unit tests, therefore we assume it is working as intended. Space is not a problem anymore, but the strategy now seems less obvious than before.

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [76]:
def kmer_index(s, k):
    kmer_ds = {}
    i = 0

    while i + k <= len(s):
        kmer = s[i:i + k]
        
        if kmer not in kmer_ds:
            kmer_ds[kmer] = []

        kmer_ds[kmer].append(i)

        i += 1                

    return kmer_ds

if __name__ == '__main__':
    k=2
    s = "ATGATATCATCGACGATGTAG"
    print("Using string:")
    print(s)
    print("".join([str(i%10) for i in range(len(s))]))
    print(f"\n{k}-mer index is:")
    d=kmer_index(s, k)
    print(dict(d))

Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{'AT': [0, 3, 5, 8, 15], 'TG': [1, 16], 'GA': [2, 11, 14], 'TA': [4, 18], 'TC': [6, 9], 'CA': [7], 'CG': [10, 13], 'AC': [12], 'GT': [17], 'AG': [19]}


### Idea of solution

The function iterates over each kmer and stores the index of each occurrence in the right dict key.

### Discussion

The function returns the correct indexes for each k-mer in the test sequence.

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [77]:
def codon_probabilities(rna):
    """
    Given an RNA sequence, simply calculates the proability of
    all 3-mers empirically based on the sequence
    """
    c_probs = {}

    for codon in product(set(rna), repeat=3):
        c_seq = "".join(codon)
        if c_seq in rna[:-1]:
            kmer = kmer_index(rna, 3)
            c_probs[c_seq] = len(kmer[c_seq]) / len(rna)
        else: 
            c_probs[c_seq] = 0 

    print(c_probs)

    return c_probs
    
def kullback_leibler(p, q):
    """
    Computes Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    The divergence is defined only when q[event] == 0 implies p[event] == 0.
    """
    div = 0.0

    for e in p:
        if p[e] == 0 or q[e] == 0:
            if p[e] == q[e]:
                pass 
            else:
                if q[e] == 0:
                    raise ZeroDivisionError
                else:
                    pass
        else:
            div += p[e] * np.log2(p[e] / q[e])

    return div

if __name__ == '__main__':
    aas = list("*ACDEFGHIKLMNPQRSTVWY") # List of amino acids
    n = 10000
    
    # generate a random protein and some associated rna
    protein = "".join(choice(aas, n))    
    rna = ProteinToRandomRNA().convert(protein)
    
    # Maybe check that converting back to protein results in the same sequence
    assert rna_to_prot(rna) == protein
    
    # Calculate codon probabilities of the rna sequence
    cp_predicted = codon_probabilities(rna) # placeholder call
    
    # Calculate codon probabilities based on the codon usage table
    cp_orig = {}
    i = 0
    table_probs = get_probabability_dict()

    while i + 3 < len(rna):
        kmer = rna[i:i + 3]
        cp_orig[kmer] = table_probs[kmer]

        i += 1
    
    # Create a completely random RNA sequence and get the codon probabilities
    rand_rna = ProteinToRandomRNA().convert(protein)
    cp_uniform = codon_probabilities(rand_rna) # placeholder call
    
    print("d(original || predicted) =", kullback_leibler(cp_orig, cp_predicted))
    print("d(predicted || original) =", kullback_leibler(cp_predicted, cp_orig))
    print()
    print("d(original || uniform) =", kullback_leibler(cp_orig, cp_uniform))
    print("d(uniform || original) =", kullback_leibler(cp_uniform, cp_orig))
    print()
    print("d(predicted || uniform) =", kullback_leibler(cp_predicted, cp_uniform))
    print("d(uniform || predicted) =", kullback_leibler(cp_uniform, cp_predicted))

    p = dict(zip("ACGT", [1.0, 0.0, 0.0, 0.0]))
    q = dict(zip("ACGT", [0.25]*4))
    print(kullback_leibler(p, q))

{'AAA': 0.019833333333333335, 'AAG': 0.016966666666666668, 'AAU': 0.0203, 'AAC': 0.016, 'AGA': 0.019133333333333332, 'AGG': 0.0134, 'AGU': 0.0143, 'AGC': 0.013166666666666667, 'AUA': 0.0151, 'AUG': 0.029, 'AUU': 0.0174, 'AUC': 0.015833333333333335, 'ACA': 0.020633333333333333, 'ACG': 0.010966666666666666, 'ACU': 0.015733333333333332, 'ACC': 0.0175, 'GAA': 0.0217, 'GAG': 0.015366666666666667, 'GAU': 0.020133333333333333, 'GAC': 0.017733333333333334, 'GGA': 0.020033333333333334, 'GGG': 0.015666666666666666, 'GGU': 0.014, 'GGC': 0.015833333333333335, 'GUA': 0.012433333333333333, 'GUG': 0.0207, 'GUU': 0.012266666666666667, 'GUC': 0.009966666666666667, 'GCA': 0.017533333333333335, 'GCG': 0.008566666666666667, 'GCU': 0.013866666666666666, 'GCC': 0.015133333333333334, 'UAA': 0.0131, 'UAG': 0.0087, 'UAU': 0.016233333333333332, 'UAC': 0.013433333333333334, 'UGA': 0.023766666666666665, 'UGG': 0.026766666666666668, 'UGU': 0.0196, 'UGC': 0.017933333333333332, 'UUA': 0.010433333333333333, 'UUG': 0.

### Idea of solution

The function `codon_probabilities()` computes the probabilities of each codon based on their frequencies inside the sequnce.
The `kullback_leiber()` module computes the divergence between two probability distributions.


### Discussion

The tests pass and we also have made sure that we do not get `log(0)`, by checking when `p[e]` is zero and simply not counting that term.

## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is

\begin{array}{l|rrrr}
 &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{array}.

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function ```get_stationary_distributions``` that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the ```main``` function print the stationary distributions
  of the above transition matrix.

In [88]:
def get_stationary_distributions(transition):
    """
    The function get a transition matrix of a degree one Markov chain as parameter.
    It returns a list of stationary distributions, in vector form, for that chain.
    """
    tt = transition.T

    w, v = np.linalg.eig(tt)
    i1 = np.isclose(w, 1)
    v1 = v[:, i1]

    v1 = v1 / v1.sum(axis=0)

    return np.abs(v1.T)
    
    
if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("\n".join(
        ", ".join(
            f"{pv:+.3f}"
            for pv in p) 
        for p in get_stationary_distributions(transition)))

+0.333, +0.000, +0.667, +0.000
+0.000, +0.250, +0.000, +0.750


### Idea of solution
The description is already pretty self explanatory. What we did is indeed transposing the transition matrix, calculate the eigenvectors associated to the unit eigenvalue, transpose them and returned.

### Discussion
The unit tests show that the proposed solution is succesfull. It is also efficient since this time we are using `numpy` which does a great job operating on matrices.

21. Implement the `kl_divergence` function below so that the main guarded code runs properly. Using your modified Markov chain generator generate a nucleotide sequence $s$ of length $10\;000$. Choose prefixes of $s$ of lengths $1, 10, 100, 1000$, and $10\;000$. For each of these prefixes find out their nucleotide distribution (of order 0) using your earlier tool. Use 1 as the pseudo count. Then, for each prefix, compute the KL divergence between the initial distribution and the normalized nucleotide distribution.

In [89]:
def kl_divergences(initial, transition):
    """
    Calculates the the Kullback-Leibler divergences between empirical distributions
    generated using a markov model seeded with an initial distributin and a transition 
    matrix, and the initial distribution.
    Sequences of length [1, 10, 100, 1000, 10000] are generated.
    """
    prefs = [1, 10, 100, 1000, 10000]
    divs = []
    zeroth = dict(zip("ACGT", initial))
    kth = {ch: dict(zip([k for k in "ACGT"], transition[i, :].tolist())) for i, ch in enumerate("ACGT")}
    mc = MarkovChain(zeroth, kth, k=1)
    seq = mc.generate(10000)

    for pref in prefs:
        probs = better_context_probabilities(seq[:pref], 0)[""]
        divs.append(kullback_leibler(zeroth, probs))

    return zip(prefs, divs)

if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("Transition probabilities are:")
    print(transition)
    stationary_distributions = get_stationary_distributions(transition)
    print("Stationary distributions:")
    print(np.stack(stationary_distributions))
    initial = stationary_distributions[1]
    print("Using [{}] as initial distribution\n".format(", ".join(f"{v:.2f}" for v in initial)))
    results = kl_divergences(initial, transition)
    for prefix_length, divergence in results: # iterate on prefix lengths in order (1, 10, 100...)
        print("KL divergence of stationary distribution prefix " \
              "of length {:5d} is {:.8f}".format(prefix_length, divergence))

Transition probabilities are:
[[0.3  0.   0.7  0.  ]
 [0.   0.4  0.   0.6 ]
 [0.35 0.   0.65 0.  ]
 [0.   0.2  0.   0.8 ]]
Stationary distributions:
[[0.33333333 0.         0.66666667 0.        ]
 [0.         0.25       0.         0.75      ]]
Using [0.00, 0.25, 0.00, 0.75] as initial distribution

KL divergence of stationary distribution prefix of length     1 is 0.76064997
KL divergence of stationary distribution prefix of length    10 is 0.55279200
KL divergence of stationary distribution prefix of length   100 is 0.08485110
KL divergence of stationary distribution prefix of length  1000 is 0.00321694
KL divergence of stationary distribution prefix of length 10000 is 0.00048261


### Idea of solution

We use the initial distribution as the `zeroth` and the transition matrix as the `kth` (where k is equal to 1).
This is what `initial` and `transition` represent in our `MarkovChain` model defined earlier. 
We are going to use that class (ignoring the performance benefits of sampling from the sequence itself), and we are going to generate a sequence of 10000 symbols using the probability given by `initial`. 
For each prefix, we calculate the probability distribution and the KL divergence between it and the initial distribution, and we append it in our list of divergences that we called `div`.

### Discussion
The tests suggest we have got it correctly. Notice how, as we increase the prefix length, `probs` and `initial` converge, i.e. the longer the sequence, the more its distribution converges with the stationary distribution.

22. Implement the following in the ```main``` function.
Find the stationary distribution for the following transition matrix:  

\begin{array}{ l | r r r r}
 & A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{array}

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.

The `main` function should return tuples, where the first element is the (random) initial distribution and the second element contains the results as a list of tuples where the first element is the kl divergence and the second element the empirical nucleotide distribution, for the different prefix lengths.

The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! That is the last line of the tables should have KL-divergence very close to $0$ and an empirical distribution very close to the equilibrium distribution.


In [90]:
def main(transition, equilibrium_distribution):
    initial = []
    divs_and_emps = []

    samp_dists = np.random.dirichlet(equilibrium_distribution, 2)

    for samp in samp_dists:
        print(samp)
        initial.append(samp)
    
    return zip(initial, divs_and_emps)


if __name__ == "__main__":
    transition = np.array([[0.3, 0.1, 0.5, 0.1],
                           [0.2, 0.3, 0.15, 0.35],
                           [0.25, 0.15, 0.2, 0.4],
                           [0.35, 0.2, 0.4, 0.05]])
    print("Transition probabilities are:", transition, sep="\n")
    stationary_distributions = get_stationary_distributions(transition)
    # Uncomment the below line to check that there actually is only one stationary distribution
    assert len(stationary_distributions) == 1
    equilibrium_distribution = stationary_distributions[0]
    print("Equilibrium distribution:")
    print(equilibrium_distribution)
    for initial_distribution, results in main(transition, equilibrium_distribution):
        print("\nUsing {} as initial distribution:".format(initial_distribution))
        print("kl-divergence   empirical distribution")
        print("\n".join("{:.11f}   {}".format(di, kl) for di, kl in results))

Transition probabilities are:
[[0.3  0.1  0.5  0.1 ]
 [0.2  0.3  0.15 0.35]
 [0.25 0.15 0.2  0.4 ]
 [0.35 0.2  0.4  0.05]]
Equilibrium distribution:
[0.27803345 0.17353238 0.32035021 0.22808396]
[9.97000252e-02 9.00291830e-01 8.14450265e-06 2.76577477e-12]
[8.92559581e-01 8.32675652e-02 2.41727388e-02 1.15163075e-07]


### Idea of solution

fill in

### Discussion
fill in